In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/language-translation-englishfrench/eng_-french.csv


In [2]:
import collections
import numpy as np
import pandas as pd
import keras
import tensorflow as tf


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential

In [3]:
data = pd.read_csv("../input/language-translation-englishfrench/eng_-french.csv")
data.tail()

,English words/sentences,French words/sentences
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
175620,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [4]:
english = data["English words/sentences"]
french = data["French words/sentences"]

In [5]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for i,text in enumerate(english):
  stri = ""
  txt = tokenizer.tokenize(text)
  for j in txt:
    j = j.lower()
    stri = stri + j
    stri = stri + " "
  english[i] = stri

In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for i,text in enumerate(french):
  stri = ""
  txt = tokenizer.tokenize(text)
  for j in txt:
    j = j.lower()
    stri = stri + j
    stri = stri + " "
  french[i] = stri

In [7]:
print(english[0:10])
print(french[0:10])

0      hi 
1     run 
2     run 
3     who 
4     wow 
5    fire 
6    help 
7    jump 
8    stop 
9    stop 
Name: English words/sentences, dtype: object
0        salut 
1        cours 
2       courez 
3          qui 
4     ça alors 
5       au feu 
6     à l aide 
7        saute 
8    ça suffit 
9         stop 
Name: French words/sentences, dtype: object


In [8]:
n1 = 9900
n2=10000
eng = []
fre = []

for data in english:
  eng.append(data)

for data in french:
  fre.append(data)

eng = np.asarray(eng)
fre = np.asarray(fre)

eng = eng[0:175000]
fre = fre[0:175000]

for i in range(n1,n2):
  print(eng[i] + "\t->\t" + fre[i] + "\n")

are you envious 	->	êtes vous envieux 

are you envious 	->	êtes vous envieuses 

are you envious 	->	êtes vous envieuse 

are you envious 	->	es tu envieux 

are you envious 	->	es tu envieuse 

are you envious 	->	es tu jaloux 

are you envious 	->	es tu jalouse 

are you envious 	->	êtes vous jaloux 

are you envious 	->	êtes vous jalouses 

are you envious 	->	êtes vous jalouse 

are you excited 	->	es tu énervé 

are you excited 	->	êtes vous énervé 

are you excited 	->	êtes vous énervés 

are you excited 	->	êtes vous énervées 

are you guys ok 	->	ça va tout le monde 

are you healthy 	->	êtes vous en bonne santé 

are you in love 	->	es tu amoureux 

are you injured 	->	êtes vous blessés 

are you injured 	->	êtes vous blessées 

are you injured 	->	êtes vous blessée 

are you injured 	->	es tu blessé 

are you injured 	->	es tu blessée 

are you injured 	->	êtes vous blessé 

are you insured 	->	êtes vous assuré 

are you insured 	->	es tu assuré 

are you jealous 	->	es tu j

In [9]:
print("English Shape is " + str(eng.shape))
print("French Shape is " + str(fre.shape))

English Shape is (175000,)
French Shape is (175000,)


In [10]:
english_word_counter = collections.Counter([word for sentence in eng for word in sentence.split(" ")])
french_word_counter = collections.Counter([word for sentence in fre for word in sentence.split(" ")])

print('{} English words.'.format(len([word for sentence in eng for word in sentence.split()])))
print('{} French words.'.format(len([word for sentence in fre for word in sentence.split()])))
print("\n")
print('{} unique English words.'.format(len(english_word_counter)))
print('{} unique French words.'.format(len(french_word_counter)))
print("\n")
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_word_counter.most_common(10)))[0]) + '"')
print("\n")
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_word_counter.most_common(10)))[0]) + '"')

1133720 English words.
1250733 French words.


13917 unique English words.
23918 unique French words.


10 Most common words in the English dataset:
"" "i" "you" "to" "the" "a" "t" "is" "that" "tom"


10 Most common words in the French dataset:
"" "je" "de" "pas" "est" "vous" "que" "il" "à" "ne"


In [11]:
def tokenize(x):
  tokenizer = Tokenizer(char_level=False,oov_token=" ")
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
  
text , tokenizer = tokenize(text_sentences)
print(text)
print(tokenizer.word_index)

[[2, 3, 5, 6, 7, 8, 2, 9, 10], [11, 12, 13, 3, 14, 15, 16, 17, 4, 18], [19, 20, 4, 21, 22]]
{' ': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, 'my': 13, 'study': 14, 'of': 15, 'lexicography': 16, 'won': 17, 'prize': 18, 'this': 19, 'is': 20, 'short': 21, 'sentence': 22}


In [12]:
def pad(x,length=None):
  if (length==None):
    length = max([len(sentence) for sentence in x])
  a = pad_sequences(x,maxlen=length,padding="post")
  return a

test_pad = pad(text)
for i, (token_sent, pad_sent) in enumerate(zip(text, test_pad)):
    print('Sequence {} in x'.format(i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10]
  Output: [ 2  3  5  6  7  8  2  9 10  0]
Sequence 2 in x
  Input:  [11 12 13  3 14 15 16 17  4 18]
  Output: [11 12 13  3 14 15 16 17  4 18]
Sequence 3 in x
  Input:  [19 20  4 21 22]
  Output: [19 20  4 21 22  0  0  0  0  0]


In [13]:
def preprocess(x,y):
  preprocess_x,x_tk = tokenize(x)
  preprocess_y,y_tk = tokenize(y)

  preprocess_x = pad(preprocess_x)
  preprocess_y = pad(preprocess_y)
  preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

  return preprocess_x, preprocess_y, x_tk, y_tk

In [14]:
pre_eng,pre_fre,eng_tk,fre_tk = preprocess(eng,fre)
max_eng_seq_len = pre_eng.shape[1]
max_fr_seq_len = pre_fre.shape[1]
english_vocab_size = len(eng_tk.word_index)
french_vocab_size = len(fre_tk.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_eng_seq_len)
print("Max French sentence length:", max_fr_seq_len)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 21
Max French sentence length: 26
English vocabulary size: 13917
French vocabulary size: 23918


In [15]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 1e-3
    model = keras.Sequential([
        Embedding(english_vocab_size+1, 128, input_length=input_shape[1]),
        Bidirectional(GRU(128, return_sequences=True)),
        tf.keras.layers.Dropout(0.25),
        TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    ])
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(pre_eng, max_fr_seq_len)
rnn_model = embed_model(tmp_x.shape,max_fr_seq_len,english_vocab_size,french_vocab_size)
rnn_model.fit(tmp_x, pre_fre, batch_size=1024, epochs=50, validation_split=0.2)
rnn_model.save_weights("rnn_model_weights.h5")
#rnn_model.load_weights("/content/rnn_model_weights.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 128)           1781504   
_________________________________________________________________
bidirectional (Bidirectional (None, 26, 256)           198144    
_________________________________________________________________
dropout (Dropout)            (None, 26, 256)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 26, 23918)         6146926   
Total params: 8,126,574
Trainable params: 8,126,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
137/137 [==============================] - 73s 535ms/step - loss: 2.8739 - accuracy: 0.7602 - val_loss: nan - val_accuracy: 0.5928
Epoch 2/50
137/137 [==============================] - 73s 533ms/step - loss: 1.5941 - accuracy: 0.7686

In [16]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [17]:
for i in range(5):
  print(logits_to_text(rnn_model.predict(tmp_x[1000])[i], fre_tk))
print(tmp_x[10000])

i = 10000
print(eng[i] + "\t->\t" + fre[i] + "\n")


#[24 ,80 ,159 ,0 ,  0 ,  0,   0,   0,   0,   0,   0]

nous
gaspillez
timides
<PAD>
<PAD>
[ 30  55 359   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
be very careful 	->	sois très prudente 



In [18]:
new_words = []
num_arr = [0 for i in range(11)]
sentence = "is"
sentence = sentence.lower()
words = sentence.split(" ")

for i,word in enumerate(words):
  num_arr[i] = eng_tk.word_index[word]


arra = []
for i in range(10):
  arra.append(logits_to_text(rnn_model.predict(num_arr)[i], fre_tk))

sent = ""
for word in arra:
  if word == "<PAD>":
    break
  else:
    sent = sent + word
    sent = sent + " "

print(sent)

fox 
